In [1]:
import cv2
import numpy as np

video_cap = cv2.VideoCapture("http://192.168.0.104:8080/video")


#--------------------
def nothing(x):
    pass
cv2.namedWindow("AdjustBar")
cv2.createTrackbar("L-H", "AdjustBar", 0, 100, nothing)
cv2.createTrackbar("L-S", "AdjustBar", 66, 255, nothing)
cv2.createTrackbar("L-V", "AdjustBar", 134, 100, nothing)
cv2.createTrackbar("U-H", "AdjustBar", 180, 180, nothing)
cv2.createTrackbar("U-S", "AdjustBar", 255, 255, nothing)
cv2.createTrackbar("U-V", "AdjustBar", 243, 255, nothing)
#--------------------

while 1:
    _, frame = video_cap.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    l_h = cv2.getTrackbarPos(trackbarname="L-H", winname="AdjustBar")
    l_s = cv2.getTrackbarPos(trackbarname="L-S", winname="AdjustBar")
    l_v = cv2.getTrackbarPos(trackbarname="L-V", winname="AdjustBar")
    u_h = cv2.getTrackbarPos(trackbarname="U-H", winname="AdjustBar")
    u_s = cv2.getTrackbarPos(trackbarname="U-S", winname="AdjustBar")
    u_v = cv2.getTrackbarPos(trackbarname="U-V", winname="AdjustBar")

    lower_red = np.array([l_h, l_s, l_v])
    upper_red = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower_red, upper_red)
    kernel = np.ones(shape=(5, 5), dtype=np.uint8)
    mask = cv2.erode(mask, kernel)

    #-----------------------
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        appox = cv2.approxPolyDP(cnt, 0.02*cv2.arcLength(cnt, True), True)
        x = appox.ravel()[0]
        y = appox.ravel()[1]

        if area > 450:
            cv2.drawContours(frame, appox, 0, (0,0,0),3)
        text = str(len(appox)) + " canh"
        cv2.putText(frame, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),1)
    
    mask = cv2.resize(mask, (mask.shape[1]//2, mask.shape[0]//2))
    frame = cv2.resize(frame, (frame.shape[1]//2, frame.shape[0]//2))
    cv2.imshow("mask", mask)
    cv2.imshow("Nhan dien", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video_cap.release()
cv2.destroyAllWindows()